In [1]:
# All imports

import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator 
import random

In [2]:
def createBB84Circuit(bitString,basisString):
    circuit = QuantumCircuit(len(bitString),len(bitString))
    for i in range(len(bitString)):
        if bitString[i] == '1':
            circuit.x(i)
        if basisString[i] == '1':
            circuit.h(i)
        circuit.measure(i,i)
    return circuit


In [3]:
def simulateCircuit(circuit):
    simulator = AerSimulator()
    optimizedCircuit = transpile(circuit,simulator)
    job = simulator.run(optimizedCircuit, shots=100000)
    counts = job.result().get_counts()
    return counts


In [4]:
#testing
v_bit_string = '1101'
v_basis_string = '1010'  
s_basis_string = '1010'    

v_circuit = createBB84Circuit(v_bit_string, v_basis_string)

s_circuit = createBB84Circuit(v_bit_string, s_basis_string)

v_counts = simulateCircuit(v_circuit)
s_counts = simulateCircuit(s_circuit)

matching_indices = [i for i in range(len(v_basis_string)) if v_basis_string[i] == s_basis_string[i]]
print("Matching basis positions:", matching_indices)

matching_bits = [v_bit_string[i] for i in matching_indices]
print("Matching bits from v:", matching_bits)


secure_key = []
for key, count in s_counts.items():
    s_bit_string = key  
    matching_s_bits = [s_bit_string[i] for i in matching_indices]
    print(f"s's bit string: {s_bit_string}, Matching s bits: {matching_s_bits}")

    current_secure_key = []
    for i, bit in enumerate(matching_bits):
        if bit == matching_s_bits[i]:
            current_secure_key.append(bit)

    if current_secure_key:
        secure_key = current_secure_key  
        break  

if secure_key:
    print("Secure key generated:", ''.join(secure_key))
else:
    print("No secure key generated. Possible eavesdropping detected.")



Matching basis positions: [0, 1, 2, 3]
Matching bits from v: ['1', '1', '0', '1']
s's bit string: 1111, Matching s bits: ['1', '1', '1', '1']
Secure key generated: 111
